### Sequence 함수에 백테스팅 추가

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [13]:
file_path = '../../data/' 
df = pd.read_csv(file_path + 'bitcoin_data_num_rows_gt_5.csv')
df = df[:10000]
df = df.sort_values(by='window_start', ascending=True) # 시간순 정렬
print(df.shape)
df.head()

(10000, 85)


,window_start,window_end,num_rows,lowest_return,highest_return,high_low_gap,trade_vol,volume_power,beginning_price,ending_price,...,ob_end_bs_14,ob_end_bias_0,ob_end_bias_1,ob_end_bias_4,ob_end_bidask_spread,ob_end_liq_0,ob_end_liq_1,ob_end_liq_4,highest_possible_return,del_idx
0,2022-12-16 21:05:30,2022-12-16 21:06:00,14,0.000000,0.000089,0.000089,1.468656,0.747351,22568000.0,22570000.0,...,1.467714,5.470422,10.649683,3.235541,2.0,0.001693,0.002198,0.002412,1.000000,0
1,2022-12-16 21:06:00,2022-12-16 21:06:30,10,0.000000,0.000089,0.000089,0.567585,0.027857,22568000.0,22570000.0,...,0.143039,4.224361,14.918538,3.856600,2.0,0.000531,0.001064,0.001471,1.000000,0
2,2022-12-16 21:06:30,2022-12-16 21:07:00,24,-0.000576,0.000044,0.000620,1.677093,0.146635,22570000.0,22570000.0,...,0.271898,17.677511,9.697905,1.106227,14.0,0.000449,0.000536,0.001821,0.999778,0
3,2022-12-16 21:07:00,2022-12-16 21:07:30,22,-0.000044,0.000443,0.000488,2.439677,0.751995,22557000.0,22567000.0,...,0.640898,95.630870,3.371113,1.367349,2.0,0.000416,0.000480,0.001422,0.999911,0
4,2022-12-16 21:07:30,2022-12-16 21:08:00,24,-0.000443,0.000000,0.000443,2.345821,-0.915608,22565000.0,22555000.0,...,0.081040,0.114815,0.828364,0.068175,10.0,0.000311,0.000560,0.003454,0.999911,0


In [14]:
"""
Note: bitcoin_prepro_v2.ipynb 파일을 통해 생성된 df에 적용할 것
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def create_sequence(df, seq_len):
    # 1. 변수 선택
    # 종속변수 리스트
    target_var_lst = ['returns', 'returns_next10m', 'realized_vol_next10m']
    target_var = 'returns_next10m' # 종속변수

    # 시퀀스 생성 전 필요없는 컬럼 삭제
    df.drop(columns=['window_start', 'window_end','num_rows', 'time_id'], inplace=True) # 시간 관련 변수

    # target을 제외한 나머지 종속변수 삭제
    cols_to_drop = [var for var in target_var_lst if var != target_var]
    df.drop(columns=cols_to_drop, inplace=True) # 종속변수
    #df['returns_next10m'] = df['returns_next10m'].apply(lambda x: 0 if x <= 0 else 1) 2진 분류화는 ipynb 파일에서 직접 수행

    # 종속변수를 데이터 프레임 맨 뒤로 옮기기
    cols = df.columns.tolist()
    cols = [col for col in cols if col != 'returns_next10m'] + ['returns_next10m'] # 종속변수 맨 뒤로
    df = df[cols]

    # 2. sequence 생성
    sequences = []
    scaler = MinMaxScaler()
    
    for start_idx in range(len(df) - seq_len + 1):  # 데이터 프레임을 순회하며 시퀀스 생성
        end_idx = start_idx + seq_len
        sequence = df.iloc[start_idx:end_idx]
        
        # 시퀀스 내에 del_idx가 1인 행이 있다면, 해당 시퀀스를 제외
        if sequence['del_idx'].sum() == 0:
            # 예측하고자 하는 마지막 피처의 값을 제외하고 스케일링
            scaled_sequence = scaler.fit_transform(sequence.drop(columns=['del_idx', target_var]))
            
            # 스케일링된 시퀀스에 예측하고자 하는 마지막 피처의 값을 추가
            scaled_sequence_with_target = pd.concat([pd.DataFrame(scaled_sequence), sequence[target_var].reset_index(drop=True)], axis=1)
            
            # 최종 시퀀스 추가
            sequences.append(scaled_sequence_with_target.values)
            
    # 3. X, y split
    sequences = np.array(sequences)
    # X와 y를 분리
    X = sequences[:, :, :-1] # 마지s막 시퀀스와 마지막 컬럼을 제외한 나머지
    y = sequences[:, -1, -1].reshape(-1, 1) # 각 시퀀스의 마지막 행, 마지막 컬럼의 값

    return X, y


In [15]:
X, y = create_sequence(df, 20)

In [16]:
print(len(X))
print(len(y))

4406
4406


In [24]:
"""
Note: bitcoin_prepro_v2.ipynb 파일을 통해 생성된 df에 적용할 것
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def createSeqForBacktest(df, seq_len):
    # 1. 변수 선택
    # 종속변수 리스트
    target_var_lst = ['returns', 'returns_next10m', 'realized_vol_next10m']
    target_var = 'returns_next10m' # 종속변수

    # 시퀀스 생성 전 필요없는 컬럼 삭제
    df.drop(columns=['window_start', 'window_end','num_rows', 'time_id'], inplace=True) # 시간 관련 변수

    # target을 제외한 나머지 종속변수 삭제
    cols_to_drop = [var for var in target_var_lst if var != target_var]
    df.drop(columns=cols_to_drop, inplace=True) # 종속변수
    #df['returns_next10m'] = df['returns_next10m'].apply(lambda x: 0 if x <= 0 else 1) 2진 분류화는 ipynb 파일에서 직접 수행

    # 종속변수를 데이터 프레임 맨 뒤로 옮기기
    cols = df.columns.tolist()
    cols = [col for col in cols if col != ['returns_next10m_binary', 'returns_next10m']] + ['returns_next10m', 'returns_next10m_binary'] # 종속변수 맨 뒤로
    df = df[cols]

    # 2. sequence 생성
    sequences = []
    scaler = MinMaxScaler()
    
    for start_idx in range(len(df) - seq_len + 1):  # 데이터 프레임을 순회하며 시퀀스 생성
        end_idx = start_idx + seq_len
        sequence = df.iloc[start_idx:end_idx]
        
        # 시퀀스 내에 del_idx가 1인 행이 있다면, 해당 시퀀스를 제외
        if sequence['del_idx'].sum() == 0:

            # 예측하고자 하는 마지막 피처의 값을 제외하고 스케일링
            scaled_sequence = scaler.fit_transform(sequence.drop(columns=['del_idx', target_var]))
            
            # 스케일링된 시퀀스에 예측하고자 하는 마지막 피처의 값을 추가
            scaled_sequence_with_target = pd.concat([pd.DataFrame(scaled_sequence), sequence[target_var].reset_index(drop=True)], axis=1)
            
            # 최종 시퀀스 추가
            sequences.append(scaled_sequence_with_target.values)
            
    # 3. X, y split
    sequences = np.array(sequences)
    # X와 y를 분리
    X = sequences[:, :, :-1] # 마지s막 시퀀스와 마지막 컬럼을 제외한 나머지
    y = sequences[:, -1, -1].reshape(-1, 1) # 각 시퀀스의 마지막 행, 마지막 컬럼의 값
    y_for_backtest = sequence[:, -1, -2].reshape(-1, 1)

    return X, y, y_for_backtest


In [28]:
def createSeqForBacktest(df, seq_len):
    # 1. 변수 선택
    target_var_lst = ['returns', 'returns_next10m', 'realized_vol_next10m']
    target_var = 'returns_next10m' # 종속변수

    # 시퀀스 생성 전 필요없는 컬럼 삭제
    df.drop(columns=['window_start', 'window_end','num_rows', 'time_id'], inplace=True)

    # target을 제외한 나머지 종속변수 삭제
    cols_to_drop = [var for var in target_var_lst if var != target_var]
    df.drop(columns=cols_to_drop, inplace=True)

    # 종속변수를 데이터 프레임 맨 뒤로 옮기기
    cols = [col for col in df.columns if col not in ['returns_next10m', 'returns_next10m_binary']] + ['returns_next10m', 'returns_next10m_binary']
    df = df[cols]

    # 2. sequence 생성
    sequences = []
    scaler = MinMaxScaler()
    
    for start_idx in range(len(df) - seq_len + 1):
        end_idx = start_idx + seq_len
        sequence = df.iloc[start_idx:end_idx]
        
        if sequence['del_idx'].sum() == 0:
            scaled_sequence = scaler.fit_transform(sequence.drop(columns=['del_idx', 'returns_next10m', 'returns_next10m_binary']))
            scaled_sequence_with_target = pd.concat([pd.DataFrame(scaled_sequence), sequence[['returns_next10m', 'returns_next10m_binary']].reset_index(drop=True)], axis=1)
            sequences.append(scaled_sequence_with_target.values)
            
    sequences = np.array(sequences)

    # 3. X, y, y_for_backtest split
    X = sequences[:, :, :-2] # 마지막 두 컬럼을 제외한 나머지
    y = sequences[:, -1, -1].reshape(-1, 1) # 각 시퀀스의 마지막 행, 마지막 컬럼
    y_for_backtest = sequences[:, -1, -2:].reshape(-1, 2) # 각 시퀀스의 마지막 행, 끝에서 두 번째와 마지막 컬럼

    return X, y, y_for_backtest


In [29]:
file_path = '../../data/' 
df = pd.read_csv(file_path + 'bitcoin_data_num_rows_gt_5.csv')
df = df[:10000]
df = df.sort_values(by='window_start', ascending=True) # 시간순 정렬
print(df.shape)
df.head()

df['returns_next10m_binary'] = df['returns_next10m'].apply(lambda x: 0 if x <= 0 else 1) # 종속변수 이진분류화


(10000, 85)


In [30]:
X, y, y_for_backtest = createSeqForBacktest(df, 20)

In [61]:
y_df = pd.DataFrame(y)
bt_df = pd.DataFrame(y_for_backtest)

y_df.merge(bt_df, how='left', )

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
4401,1.0
4402,1.0
4403,1.0
4404,0.0
